<a href="https://colab.research.google.com/drive/1R6fYYtk_kTUrFj_WWCskLl954BgZHkQP?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Hybrid (Ensamble) Search

Hybrid Search is a technique that combines multiple search methods to improve retrieval performance. Typically, it combines traditional keyword-based search (like BM25) with semantic search (using embeddings). This approach can provide better results than either method alone, especially for queries that have both keyword-specific and semantic aspects.

### Hybrid-search RAG Implementation:

1. **Hybrid Retrieval:** We use the EnsembleRetriever to get relevant documents using both keyword-based and semantic search.
2. **Response Generation:** Using the retrieved context, we generate a final response to the original query.
3. **Hybrid Search Explanation:** We generate an explanation of how the hybrid search process might have improved the retrieval of relevant information.

# Setup

1. **[LLM](https://groq.com/):** Groq's free Open source LLM endpoints([Groq API Key](https://console.groq.com/keys))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**

# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-chroma==0.2.2 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1 \
     rank_bm25==0.2.2

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [2]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [3]:
import getpass
import os

#### Provide a Groq API key. You can create one to access free open-source models at the following link.

[Groq API Creation Link](https://console.groq.com/keys)




In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


### Provide Huggingface API Key. You can create Huggingface API key at following lin

[Higgingface API Creation Link](https://huggingface.co/settings/tokens)




In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


### Step 1: Load and preprocess data code

In [6]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

### Step 2: Create vector store and BM25 retriever

In [7]:
def create_retrievers(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    vectorstore = Chroma.from_documents(chunks, embeddings)
    bm25_retriever = BM25Retriever.from_documents(chunks)
    bm25_retriever.k = 5  # Set number of documents to retrieve

    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vectorstore.as_retriever(search_kwargs={"k": 5})],
        weights=[0.5, 0.5]
    )

    return ensemble_retriever

### Helper function for safe LLM calls

In [8]:
def safe_llm_call(prompt, **kwargs):
    try:
        response = llm.invoke(prompt.format(**kwargs))
        return response.content if response else "No response generated."
    except Exception as e:
        print(f"Error in LLM call: {e}")
        return "An error occurred while generating the response."

### Step 3: Hybrid-search RAG related code

1. **Hybrid Retrieval:** We use the EnsembleRetriever to get relevant documents using both keyword-based and semantic search.
2. **Response Generation:** Using the retrieved context, we generate a final response to the original query.
3. **Hybrid Search Explanation:** We generate an explanation of how the hybrid search process might have improved the retrieval of relevant information.

In [9]:
def hybrid_search_rag(query, ensemble_retriever, llm):
    # Hybrid retrieval
    retrieved_docs = ensemble_retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Generate response
    response_prompt = PromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information retrieved using a hybrid search method combining keyword-based and semantic search. "
        "Please provide a comprehensive answer to the question, using the context when relevant "
        "and your general knowledge when necessary.\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n"
        "Answer:"
    )
    final_answer = safe_llm_call(response_prompt, context=context, query=query)

    # Generate explanation of hybrid search process
    explanation_prompt = PromptTemplate.from_template(
        "Explain how the hybrid search process, combining keyword-based and semantic search, "
        "might have improved the retrieval of relevant information for answering the given query. "
        "Consider the potential benefits of this approach compared to using only one search method.\n\n"
        "Query: {query}\n"
        "Explanation:"
    )
    hybrid_search_explanation = safe_llm_call(explanation_prompt, query=query)

    return {
        "query": query,
        "final_answer": final_answer,
        "hybrid_search_explanation": hybrid_search_explanation,
        "retrieved_context": context
    }

### Step 4: Create chunk of web data to Chroma Vector Store

In [ ]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5
)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create ensemble retriever
ensemble_retriever = create_retrievers(chunks)

### Step 5: Run Hybrid-Search RAG

This implementation shows the key parts of Hybrid-search RAG:

1. Combination of keyword-based (BM25) and semantic (vector) search for retrieval
2. Use of EnsembleRetriever to balance between different retrieval methods
3. Generation of a response using the hybrid-retrieved context
4. Explanation of the potential benefits of the hybrid search process

In [ ]:
# Example queries
queries = [
        "What are the main applications of artificial intelligence in healthcare?",
        "Explain the concept of machine learning and its relationship to AI.",
        "Discuss the ethical implications of AI in decision-making processes."
    ]

# Run Hybrid-search RAG for each query
for query in queries:
  print(f"\nQuery: {query}")
  result = hybrid_search_rag(query, ensemble_retriever, llm)
  print("Final Answer:")
  print(result["final_answer"])
  print("\nHybrid Search Explanation:")
  print(result["hybrid_search_explanation"])
  print("\nRetrieved Context (first 300 characters):")
  print(result["retrieved_context"][:300] + "...")


Query: What are the main applications of artificial intelligence in healthcare?
Final Answer:
Based on the provided context, the main applications of artificial intelligence in healthcare include:

1. **Medical Diagnosis**: AI has the potential to increase patient care and quality of life by more accurately diagnosing and treating patients, as highlighted in the Hippocratic Oath.
2. **Medical Research**: AI is an important tool for processing and integrating big data, particularly in organoid and tissue engineering development, which uses microscopy imaging as a key technique in fabrication.
3. **Processing and Integrating Big Data**: AI can overcome discrepancies in funding allocated to different fields of research, deepen the understanding of biomedically relevant pathways, and provide insights into complex biological systems.
4. **Predictive Analysis**: AI can be used to predict the result of judicial decisions, foreign policy, and supply chain management, which can have implicatio